In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import mercury as mr
from dateutil.relativedelta import relativedelta

In [3]:
import plotly as py
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import warnings
warnings.simplefilter(action= 'ignore', category=DeprecationWarning)

In [4]:
app = mr.App(title="📈 Stock Price Dashboard", description="Dashboard with financial data", show_code=False)

mercury.App

In [5]:
ticker = mr.Select(label="Please select ticker", value='NVDA', 
                   choices=['NVDA', 'INTC', 'AMD', 'TSM', 'MU'])

mercury.Select

In [6]:
mr.Md(f"# Selected ticker: {ticker.value}")

# Selected ticker: NVDA

In [7]:
period = mr.Numeric(label="Past Month(s)", value=3, min=1, max=12)

mercury.Numeric

In [14]:
# Download NVIDIA Corporation (NVDA) stock data for analysis
ticker_symbol = ticker.value # e.g., 'NVDA', 'AMD'
stock_data = yf.download(ticker_symbol, start=date.today() - relativedelta(months=+period.value), end=date.today())

# If columns are MultiIndex like ('Close','NVDA'), drop the ticker level
if isinstance(stock_data.columns, pd.MultiIndex):
    stock_data = stock_data.xs(ticker_symbol, axis=1, level=1)
    # Now columns should be: Index(['Open','High','Low','Close','Adj Close','Volume'], dtype='object')

[*********************100%***********************]  1 of 1 completed


In [10]:
# Pick the right price column
price_col = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'


In [15]:
# Create and display an interactive line plot of NVDA's Adjusted Close Prices
import plotly.express as px

# Reset index if 'Date' is the index
stock_data_reset = stock_data.reset_index()

fig = px.line(
    stock_data_reset,
    x='Date',
    y=price_col,
    title=f'{ticker_symbol} {price_col} Price',
    line_shape='linear'
)
fig.show()

In [17]:
# Create and display an interactive filled area plot of NVDA's Adjusted Close Prices
area_filled = px.area(stock_data, x=stock_data.index, y=price_col, 
              title='Adjusted Close (Filled Area)', 
              color_discrete_sequence=['green'])
area_filled.show()

In [19]:
# Create and display an interactive line plot of NVDA's Returns with a best-fit line
stock_data['Returns'] = stock_data[price_col].pct_change()

# Create a scatter plot with a trendline (best-fit line)
returns_plot = px.scatter(stock_data, x=stock_data.index, y='Returns',
                          title='Returns with Best-Fit Line',
                          trendline='ols',  # Add Ordinary Least Squares (OLS) trendline
                          color_discrete_sequence=['green'])

# Customize the trendline color by manually updating the figure
returns_plot.update_traces(marker=dict(color='green'))  # Change point color
returns_plot.data[1].line.color = 'black'  # Change trendline color

# Show the plot
returns_plot.show()

In [20]:
# Create a Quantitative Figure (QuantFig) for the NVDA stock data
import plotly.graph_objects as go
import pandas as pd

# Assuming stock_data is a DataFrame with price_col and 'Volume' columns
ticker = 'NVDA'  # Change this to your actual ticker

# Calculate Moving Averages
stock_data['SMA_10'] = stock_data[price_col].rolling(window=10).mean()
stock_data['SMA_20'] = stock_data[price_col].rolling(window=20).mean()

# Calculate Bollinger Bands
stock_data['Upper Band'] = stock_data['SMA_20'] + (stock_data[price_col].rolling(window=20).std() * 2)
stock_data['Lower Band'] = stock_data['SMA_20'] - (stock_data[price_col].rolling(window=20).std() * 2)

# Drop rows with NaN values (optional)
stock_data = stock_data.dropna()

# Create the figure
qf = go.Figure()

# Add Adjusted Close line
qf.add_trace(go.Scatter(x=stock_data.index, y=stock_data[price_col],
                         mode='lines', name=price_col))

# Add Moving Averages
qf.add_trace(go.Scatter(x=stock_data.index, y=stock_data['SMA_10'],
                         mode='lines', name='SMA 10', line=dict(color='green')))
qf.add_trace(go.Scatter(x=stock_data.index, y=stock_data['SMA_20'],
                         mode='lines', name='SMA 20', line=dict(color='lightgreen')))

# Add Bollinger Bands
qf.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Upper Band'],
                         mode='lines', name='Upper Band', line=dict(color='red')))
qf.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Lower Band'],
                         mode='lines', name='Lower Band', line=dict(color='red')))

# Add volume bars
qf.add_trace(go.Bar(x=stock_data.index, y=stock_data['Volume'], name='Volume', 
                     yaxis='y2', opacity=0.3))

# Update layout
qf.update_layout(title=f'Quantitative Figure for {ticker}',
                  yaxis_title='Price',
                  yaxis2=dict(title='Volume', overlaying='y', side='right'),
                  legend=dict(orientation='h', x=0, y=1.1))

# Show the plot
qf.show()


In [21]:
# Download stock data for semiconductor companies
semiconductor_tickers = ['NVDA', 'INTC', 'AMD', 'TSM', 'MU']
semiconductor_data = yf.download(semiconductor_tickers, start=date.today() - relativedelta(months=+period.value), end=date.today())
semiconductor_data

[*********************100%***********************]  5 of 5 completed


Price            Close                                                 \
Ticker             AMD       INTC          MU        NVDA         TSM   
Date                                                                    
2025-08-18  176.139999  23.660000  123.472679  181.999725  240.650803   
2025-08-19  166.550003  25.309999  121.973618  175.630096  231.968185   
2025-08-20  165.199997  23.540001  117.136642  175.390106  227.881088   
2025-08-21  163.710007  23.500000  115.717537  174.970123  226.615082   
2025-08-22  167.759995  24.799999  117.606354  177.979965  232.257278   
...                ...        ...         ...         ...         ...   
2025-11-11  237.520004  37.880001  241.110001  193.160004  291.170013   
2025-11-12  258.890015  37.889999  244.899994  193.800003  290.619995   
2025-11-13  247.960007  35.910000  236.949997  186.860001  282.200012   
2025-11-14  246.809998  35.520000  246.830002  190.169998  284.820007   
2025-11-17  240.520004  34.709999  241.949997  186.600006  282.010010   

Price             High                                                 ...  \
Ticker             AMD       INTC          MU        NVDA         TSM  ...   
Date                                                                   ...   
2025-08-18  178.800003  24.900000  123.602595  182.929681  242.096240  ...   
2025-08-19  173.169998  26.530001  124.162242  182.489710  239.414694  ...   
2025-08-20  166.649994  24.930000  118.835583  175.990078  228.309729  ...   
2025-08-21  165.880005  23.570000  117.656325  176.890013  229.605647  ...   
2025-08-22  168.529999  25.230000  120.624460  178.579922  233.712678  ...   
...                ...        ...         ...         ...         ...  ...   
2025-11-11  248.460007  38.220001  249.990005  195.419998  294.920013  ...   
2025-11-12  263.510010  38.200001  249.190002  195.889999  294.869995  ...   
2025-11-13  259.630005  37.310001  249.500000  191.440002  289.500000  ...   
2025-11-14  253.440002  36.099998  255.169998  191.009995  286.859985  ...   
2025-11-17  248.770004  35.730000  260.579987  189.000000  286.529999  ...   

Price             Open                                                 \
Ticker             AMD       INTC          MU        NVDA         TSM   
Date                                                                    
2025-08-18  176.759995  24.000000  121.274051  180.589816  239.235270   
2025-08-19  173.100006  24.940001  122.863060  182.419707  239.265172   
2025-08-20  164.100006  24.900000  118.625708  175.160123  227.422528   
2025-08-21  165.860001  23.530001  116.926779  174.840141  227.432495   
2025-08-22  162.169998  23.650000  117.576374  172.600264  227.282966   
...                ...        ...         ...         ...         ...   
2025-11-11  241.660004  38.150002  248.339996  195.160004  292.059998   
2025-11-12  253.130005  38.060001  244.899994  195.720001  293.700012   
2025-11-13  251.899994  37.270000  243.600006  191.050003  289.170013   
2025-11-14  240.110001  35.029999  231.449997  182.860001  275.910004   
2025-11-17  242.750000  35.450001  246.919998  185.970001  281.440002   

Price          Volume                                            
Ticker            AMD       INTC        MU       NVDA       TSM  
Date                                                             
2025-08-18   35937500  212515600  12208800  132008000   6919800  
2025-08-19   64455000  297659200  11491200  185229200  14594700  
2025-08-20   60233200  161149400  25795000  215142700  17165200  
2025-08-21   37880500   97718300  13174900  140040900   7449100  
2025-08-22   43998600  200266800  17038300  172789400  10299500  
...               ...        ...       ...        ...       ...  
2025-11-11   61336800   53741900  22622500  176483300   9666300  
2025-11-12  108942000   53876000  17263100  154935300   9715900  
2025-11-13   63178600   95593200  32324600  207423100  13827600  
2025-11-14   47655600   71716200  36171100  186591900  11486800  
202

In [22]:
# Create and display an interactive line plot of Adjusted Close Prices for semiconductor companies
newclosing = go.Figure()
newclosing.add_trace(go.Scatter(x=semiconductor_data.index, 
                         y=semiconductor_data[price_col], 
                         mode='lines', 
                         name='Adjusted Close', 
                         line=dict(color='blue')))
newclosing.update_layout(title='Semiconductor Adjusted Close Prices',
                  xaxis_title='Dates',
                  yaxis_title='Adjusted Close Prices',
                  template='plotly_white')
newclosing.show()

In [23]:
semiconductor_returns = semiconductor_data[price_col].pct_change()  # Calculate daily returns
semiconductor_returns.iloc[0, :] = 0  # Set the initial value of returns to 0 (first day)

# Define the weights for each company in the portfolio
weights = np.array([0.1, 0.2, 0.25, 0.25, 0.2])  # Adjust these weights as needed

# Calculate the weighted returns for each company
weighted_returns = weights * semiconductor_returns

# Calculate cumulative returns for the portfolio
cumulative_returns = (weighted_returns + 1).cumprod()

# Create the figure
fig = go.Figure()

# Add cumulative returns line plot
fig.add_trace(go.Scatter(x=cumulative_returns.index, 
                         y=cumulative_returns.values, 
                         mode='lines', 
                         name='Cumulative Returns', 
                         line=dict(color='blue')))

# Update layout
fig.update_layout(title='Cumulative Semiconductor Returns',
                  xaxis_title='Dates',
                  yaxis_title='Cumulative Returns',
                  template='plotly_white')

# Show the plot
fig.show()
